In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
count_parameters(model)

669706

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309648  [    0/60000]
loss: 2.292466  [ 6400/60000]
loss: 2.277254  [12800/60000]
loss: 2.266875  [19200/60000]
loss: 2.249865  [25600/60000]
loss: 2.229623  [32000/60000]
loss: 2.223492  [38400/60000]
loss: 2.196262  [44800/60000]
loss: 2.199239  [51200/60000]
loss: 2.160072  [57600/60000]
Test Error: 
 Accuracy: 46.6%, Avg loss: 2.157111 

Epoch 2
-------------------------------
loss: 2.176044  [    0/60000]
loss: 2.160685  [ 6400/60000]
loss: 2.104154  [12800/60000]
loss: 2.115921  [19200/60000]
loss: 2.063672  [25600/60000]
loss: 2.015175  [32000/60000]
loss: 2.023758  [38400/60000]
loss: 1.954054  [44800/60000]
loss: 1.964062  [51200/60000]
loss: 1.871597  [57600/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.881877 

Epoch 3
-------------------------------
loss: 1.926156  [    0/60000]
loss: 1.890675  [ 6400/60000]
loss: 1.771225  [12800/60000]
loss: 1.803295  [19200/60000]
loss: 1.695147  [25600/60000]
loss: 1.658384  [32000/600